In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import lightgbm as lgb

RANDOM_STATE = 12345

## Обзор данных

In [1]:
# Открываем файл с данными
file_path = '/datasets/users_behavior.csv'
df = pd.read_csv(file_path)

# Изучаем первые несколько строк данных
print("Первые несколько строк данных:")
display(df.head())

# Выводим общую информацию о данных
print("\nОбщая информация о данных:")
df.info()

# Описательная статистика по данным
print("\nОписательная статистика данных:")
display(df.describe().T)

# Проверка на наличие пропущенных значений
missing_values_count = df.isnull().sum().sum()
print("\nКоличество пропущенных значений в данных:", missing_values_count)

# Проверка на наличие дубликатов
duplicate_count = df.duplicated().sum()
print("\nКоличество дубликатов в данных:", duplicate_count)

NameError: name 'pd' is not defined

**Вывод:**

1. Датасет содержит 3214 записей и 5 столбцов.

Каждый объект в наборе данных представляет собой информацию о поведении одного пользователя за один месяц. В каждой строке содержатся следующие данные:

- `calls` — количество звонков, сделанных пользователем.
- `minutes` — суммарная длительность звонков в минутах.
- `messages` — количество отправленных SMS-сообщений.
- `mb_used` — объем использованного интернет-трафика в мегабайтах.
- `is_ultra` — индикатор используемого тарифа (1 — тариф "Ультра", 0 — тариф "Смарт").

2. Пропущенные значения отсутствуют, что означает, что все данные заполнены.
3. Дубликаты также отсутствуют, что указывает на то, что все записи уникальны.
4. Основные статистические характеристики данных:
   - В среднем пользователи совершают 63 звонка, разговаривают 438 минут, отправляют 38 сообщений и используют 17207 Мб интернет-трафика в месяц.
   - Значения показателей варьируются в широких пределах: от 0 до 244 звонков, от 0 до 1632 минут разговоров, от 0 до 224 сообщений и от 0 до 49745 Мб интернет-трафика.
   - Примерно 30.65% пользователей используют тариф "Ультра", остальные 69.35% — тариф "Смарт".

Эти данные можно использовать для дальнейшего анализа и построения модели машинного обучения для рекомендации тарифов.


## Разделение данных на выборки

In [3]:
# Определение признаков (X) и целевой переменной (y)
X = df.drop('is_ultra', axis=1)
y = df['is_ultra']

# Разделение данных на обучающую и тестовую выборки 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

# Разделение обучающей выборки на обучающую и валидационную 
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE, stratify=y_train)

# Вычисление размеров выборок
train_size = X_train.shape[0]
valid_size = X_valid.shape[0]
test_size = X_test.shape[0]
total_size = X.shape[0]

# Вычисление процентного соотношения
train_percentage = (train_size / total_size) * 100
valid_percentage = (valid_size / total_size) * 100
test_percentage = (test_size / total_size) * 100

# Вывод информации
print(f"Размер обучающей выборки: {train_size} ({train_percentage:.2f}%)")
print(f"Размер валидационной выборки: {valid_size} ({valid_percentage:.2f}%)")
print(f"Размер тестовой выборки: {test_size} ({test_percentage:.2f}%)")

Размер обучающей выборки: 1928 (59.99%)
Размер валидационной выборки: 643 (20.01%)
Размер тестовой выборки: 643 (20.01%)


In [4]:
# Подсчет количества значений в целевой переменной
train_class_counts = y_train.value_counts(normalize=True)
valid_class_counts = y_valid.value_counts(normalize=True)
test_class_counts = y_test.value_counts(normalize=True)

# Форматированный вывод баланса классов
print("Баланс классов в обучающей выборке:")
print(f"- Тариф \"Смарт\": {train_class_counts[0]*100:.2f}%")
print(f"- Тариф \"Ультра\": {train_class_counts[1]*100:.2f}%\n")

print("Баланс классов в валидационной выборке:")
print(f"- Тариф \"Смарт\": {valid_class_counts[0]*100:.2f}%")
print(f"- Тариф \"Ультра\": {valid_class_counts[1]*100:.2f}%\n")

print("Баланс классов в тестовой выборке:")
print(f"- Тариф \"Смарт\": {test_class_counts[0]*100:.2f}%")
print(f"- Тариф \"Ультра\": {test_class_counts[1]*100:.2f}%")

Баланс классов в обучающей выборке:
- Тариф "Смарт": 69.35%
- Тариф "Ультра": 30.65%

Баланс классов в валидационной выборке:
- Тариф "Смарт": 69.36%
- Тариф "Ультра": 30.64%

Баланс классов в тестовой выборке:
- Тариф "Смарт": 69.36%
- Тариф "Ультра": 30.64%


**Вывод:**

Данные были разделены на обучающую и тестовую выборки (соотношение 3:1:1) с использованием параметра stratify, что гарантировало сохранение пропорций классов в этих выборках.

## Подбор и оценка моделей с различными гиперпараметрами

In [5]:
# Используемые модели
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=RANDOM_STATE),
    "Random Forest": RandomForestClassifier(random_state=RANDOM_STATE),
    "Gradient Boosting": GradientBoostingClassifier(random_state=RANDOM_STATE),
    "LightGBM": lgb.LGBMClassifier(random_state=RANDOM_STATE),
    "CatBoost": CatBoostClassifier(random_state=RANDOM_STATE, verbose=0)
}

In [6]:
# Определение сетки гиперпараметров для каждой модели
param_grids = {
    "Decision Tree": {
        "max_depth": [5, 10, 15],
        "min_samples_split": [5, 10, 15],
        "class_weight": ['balanced', None]
    },
    "Random Forest": {
        "n_estimators": [50, 100, 150],
        "max_depth": [5, 10, 15],
        "min_samples_split": [5, 10, 15],
        "class_weight": ['balanced', None]
    },
    "Gradient Boosting": {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.01, 0.1, 0.5],
        "max_depth": [3, 5, 10],
    },
    "LightGBM": {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.01, 0.1, 0.5],
        "max_depth": [3, 5, 10],
        "scale_pos_weight": [1, 3, 5]
    },
    "CatBoost": {
        "iterations": [50, 100, 200],
        "learning_rate": [0.01, 0.1, 0.5],
        "depth": [3, 5, 10],
        "scale_pos_weight": [1, 3, 5]
    }
}

In [7]:
# Определение кросс-валидации
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Создание словаря для хранения лучших моделей
best_models = {}

# GridSearchCV с StratifiedKFold и оценка на валидационной выборке
best_model_name = None
best_accuracy = 0

print("\nПодбор гиперпараметров и оценка моделей на валидационной выборке с помощью GridSearchCV:\n")
for model_name, model in models.items():
    grid_search = GridSearchCV(model, param_grid=param_grids[model_name], scoring='accuracy', cv=cv)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    best_models[model_name] = best_model
    
    print(f"Модель: {model_name}")
    print(f"Лучшие параметры: {grid_search.best_params_}")
    
    y_valid_pred = best_model.predict(X_valid)
    valid_accuracy = accuracy_score(y_valid, y_valid_pred)
    
    if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_model_name = model_name
    
    print(f"Accuracy: {valid_accuracy:.4f}\n")


Подбор гиперпараметров и оценка моделей на валидационной выборке с помощью GridSearchCV:

Модель: Decision Tree
Лучшие параметры: {'class_weight': None, 'max_depth': 5, 'min_samples_split': 5}
Accuracy: 0.8134

Модель: Random Forest
Лучшие параметры: {'class_weight': None, 'max_depth': 10, 'min_samples_split': 15, 'n_estimators': 150}
Accuracy: 0.8243

Модель: Gradient Boosting
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Accuracy: 0.8149

Модель: LightGBM
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'scale_pos_weight': 1}
Accuracy: 0.8149

Модель: CatBoost
Лучшие параметры: {'depth': 3, 'iterations': 200, 'learning_rate': 0.1, 'scale_pos_weight': 1}
Accuracy: 0.8180



In [8]:
# Дообучение лучшей модели на объединенной выборке
final_best_model = best_models[best_model_name]

X_train_valid = pd.concat([X_train, X_valid])
y_train_valid = pd.concat([y_train, y_valid])

final_best_model.fit(X_train_valid, y_train_valid)

RandomForestClassifier(max_depth=10, min_samples_split=15, n_estimators=150,
                       random_state=12345)

## Оценка качества модели на тестовой выборке

In [9]:
# Оценка лучшей модели на тестовой выборке
y_test_pred = final_best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Accuracy на тестовой выборке: {test_accuracy:.4f}")

Accuracy на тестовой выборке: 0.8227


**Вывод:**

Итоговая модель достигла значения accuracy на тестовой выборке, равного 0.8227. Это значение превышает установленный порог в 0.75, что указывает на успешное выполнение поставленной задачи.

## Проверка модели на адекватность

In [10]:
# Создание и обучение DummyClassifier с параметром strategy='most_frequent'
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(X_train, y_train)

# Оценка качества модели DummyClassifier на тестовой выборке
dummy_accuracy = dummy_clf.score(X_test, y_test)
print(f"Accuracy модели DummyClassifier на тестовой выборке: {dummy_accuracy:.4f}")

Accuracy модели DummyClassifier на тестовой выборке: 0.6936


**Вывод:**

Для проверки адекватности модели была использована базовая модель DummyClassifier, которая показала accuracy на тестовой выборке равное 0.6936. Данное значение значительно ниже, чем accuracy итоговой модели (0.8227). Это свидетельствует о том, что наша модель действительно извлекает полезные закономерности из данных и делает осмысленные предсказания, превосходя простую модель, которая делает случайные предсказания.

## Общий вывод

Целью проекта было разработать систему, способную проанализировать поведение клиентов мобильного оператора и предложить им новый тариф: «Смарт» или «Ультра». Для успешного завершения проекта требовалось построить модель с долей правильных ответов не менее 75%.

Мы сравнили несколько моделей машинного обучения для выбора лучшего тарифа для пользователей мобильного оператора. Были использованы следующие модели: Decision Tree, Random Forest, Gradient Boosting, LightGBM и CatBoost.

**Random Forest** оказалась лучшей моделью с точностью около **82.2%** на тестовой выборке, что значительно выше по сравнению с базовым уровнем (предположительно около 69.4%, представленного DummyClassifier).

### Рекомендации

Рекомендуется использовать модель Random Forest для системы предложения тарифов, поскольку она обеспечивает высокую точность и надежность в прогнозировании. Эта модель может помочь оператору мобильной связи улучшить удовлетворенность клиентов за счет предложения наиболее подходящих тарифных планов.